In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import sys
sys.path.insert(0,'./data')

import time
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
from torch import nn
from experiment.utils import *
#from models.decoder_with_attention import DecoderWithAttention
from models.decoder_with_attention_final import DecoderWithAttention
from transformers import (WEIGHTS_NAME, BertConfig,
                                  BertForSequenceClassification, BertTokenizer,
                                  )



I1201 13:45:05.513083 139878915139392 file_utils.py:39] PyTorch version 0.4.1 available.


In [2]:
def _get_available_devices():
    from tensorflow.python.client import device_lib
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(_get_available_devices())

['/device:CPU:0', '/device:XLA_CPU:0', '/device:XLA_GPU:0', '/device:GPU:0']


In [3]:
from load_datasets_final import CaptionDataset
from create_inputs_utils import (CaptionProcessor,)
import create_inputs_utils_test

#from data.load_datasets_final import *
# from data.create_inputs_utils import *

from experiment._train_one_epoch import *
from experiment._validation_one_epoch import *

I1201 13:45:06.667576 139878915139392 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
I1201 13:45:06.668815 139878915139392 configuration_utils.py:168] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "sst-2",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

I1201 13:45:07.610342 139878915139392 tokeniza

In [4]:
# Data parameters
data_folder = 'preprocessed_data'  # folder with data files saved by create_input_files.py
data_name = 'preprocessed_coco'  # base name shared by data files

# Model parameters
emb_dim = 768 #1024  # dimension of word embeddings > change bert embedding size (768)
attention_dim = 384  # dimension of attention linear layers
decoder_dim = 384  # dimension of decoder RNN
dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
cudnn.benchmark = True  # set to true only if inputs to model are fixed size; otherwise lot of computational overhead

# Training parameters
start_epoch = 0
epochs = 50  # number of epochs to train for (if early stopping is not triggered)
epochs_since_improvement = 0  # keeps track of number of epochs since there's been an improvement in validation BLEU
batch_size = 20
workers = 0 # for data-loading; right now, only 1 works with h5py
best_bleu4 = 0.  # BLEU-4 score right now
# print_freq = 100  # print training/validation stats every __ batches
checkpoint = 'ckpt/BERT_3.pth.tar'  # path to checkpoint, None if none

In [5]:
#checkpoint = None  # path to checkpoint, None if none
BERT_VOCA_SIZE = 30522

In [6]:
# Read word map
#word_map_file = os.path.join(data_folder, 'WORDMAP_' + data_name + '.json')
#with open(word_map_file, 'r') as j:
#    word_map = json.load(j)

# Initialize / load checkpoint
if checkpoint is None:
    decoder = DecoderWithAttention(attention_dim=attention_dim,
                                   embed_dim=emb_dim,
                                   decoder_dim=decoder_dim,
                                   vocab_size=BERT_VOCA_SIZE,
                                   dropout=dropout)

    decoder_optimizer = torch.optim.Adamax(params=filter(lambda p: p.requires_grad, decoder.parameters()))

else:
    checkpoint = torch.load(checkpoint)
    start_epoch = checkpoint['epoch'] + 1
    epochs_since_improvement = checkpoint['epochs_since_improvement']
    best_bleu4 = checkpoint['bleu-4']
    decoder = checkpoint['decoder']
    decoder_optimizer = checkpoint['decoder_optimizer']
    
    
# Move to GPU, if available
decoder = decoder.to(device)

# Loss functions
criterion_ce = nn.CrossEntropyLoss().to(device)
criterion_dis = nn.MultiLabelMarginLoss().to(device)

# Custom dataloaders suffle 빼버림
train_loader = torch.utils.data.DataLoader(
    CaptionDataset(data_folder, data_name, 'TRAIN'),
    batch_size=batch_size,shuffle=False,num_workers=workers, pin_memory=True)
val_loader = torch.utils.data.DataLoader(
    CaptionDataset(data_folder, data_name, 'VAL'),
    batch_size=batch_size,shuffle=False,num_workers=workers, pin_memory=True)

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:425: SourceChangeWarning: source code of class 'models.decoder_with_attention_final.DecoderWithAttention' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:425: SourceChangeWarning: source code of class 'transformers.modeling_bert.BertModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [7]:
for epoch in range(start_epoch, epochs):
    
    # Decay learning rate if there is no improvement for 8 consecutive epochs, and terminate training after 20
    if epochs_since_improvement == 20:
        break
    if epochs_since_improvement > 0 and epochs_since_improvement % 8 == 0:
        adjust_learning_rate(decoder_optimizer, 0.8)

    # One epoch's training
    train(train_loader=train_loader,
          decoder=decoder,
          criterion_ce = criterion_ce,
          criterion_dis=criterion_dis,
          decoder_optimizer=decoder_optimizer,
          epoch=epoch)


    # One epoch's validation
    recent_bleu4 = validate(val_loader=val_loader,
                            decoder=decoder,
                            criterion_ce=criterion_ce,
                            criterion_dis=criterion_dis,)
    
    # Check if there was an improvement
    is_best = recent_bleu4 > best_bleu4
    best_bleu4 = max(recent_bleu4, best_bleu4)
    if not is_best:
        epochs_since_improvement += 1
        print("\nEpochs since last best performance: %d\n" % (epochs_since_improvement,))
    else:
        epochs_since_improvement = 0

    # Save checkpoint
    save_checkpoint(data_name, epoch, epochs_since_improvement, decoder,decoder_optimizer, recent_bleu4, is_best)

Epoch: [3][0/28322]	Batch Time 0.227 (0.227)	Data Load Time 0.009 (0.009)	Loss 40.4218 (40.4218)	Top-5 Accuracy 43.515 (43.515)
Epoch: [3][100/28322]	Batch Time 0.160 (0.168)	Data Load Time 0.005 (0.012)	Loss 4.8236 (12.5110)	Top-5 Accuracy 80.083 (61.759)
Epoch: [3][200/28322]	Batch Time 0.159 (0.168)	Data Load Time 0.005 (0.012)	Loss 4.1100 (10.9873)	Top-5 Accuracy 75.877 (67.896)
Epoch: [3][300/28322]	Batch Time 0.179 (0.171)	Data Load Time 0.009 (0.015)	Loss 13.7990 (10.1687)	Top-5 Accuracy 69.478 (71.285)
Epoch: [3][400/28322]	Batch Time 0.151 (0.171)	Data Load Time 0.013 (0.014)	Loss 5.5907 (9.5902)	Top-5 Accuracy 83.190 (73.187)
Epoch: [3][500/28322]	Batch Time 0.144 (0.172)	Data Load Time 0.010 (0.015)	Loss 12.9849 (9.3125)	Top-5 Accuracy 71.053 (74.566)
Epoch: [3][600/28322]	Batch Time 0.150 (0.172)	Data Load Time 0.010 (0.015)	Loss 8.4412 (8.9286)	Top-5 Accuracy 80.508 (75.804)


KeyboardInterrupt: 